In [12]:
#pip install docplex
#Instalar en la terminal 
#pip install openpyxl
#pip install cplex
#pip install suds-py3

In [1]:
# Importar librerías
import pandas as pd
import numpy as np
from tkinter import *
from tkinter import ttk
from tkinter import simpledialog
from docplex.mp.model import Model
import cplex

In [ ]:
# Importar base de datos
bd = pd.read_excel("basedatoscelulares.xlsx")
print(bd) 

In [4]:
# Establecer presupuesto
ppto = simpledialog.askfloat(title="Presupuesto", prompt="Digite su presupuesto en pesos")
TRM = simpledialog.askfloat(title="TRM", prompt="Digite la TRM actual")

In [5]:
#Inicializar el modelo
m = Model(name="Optimización de compra de celulares")

In [6]:
#Añadir variables de decisión
ref = bd["Ref"]
n = len(ref)
variables = [m.integer_var(name=ref[i]) for i in range(n)]
#print(variables)

In [7]:
apple = []
samg = []
moto = []

for i in range(n):
    if variables[i].name == ("i"+str(i+1)):
        apple.append(variables[i])
    elif variables[i].name == ("s"+str(i+1)):
        samg.append(variables[i])
    else:
        moto.append(variables[i])

    

In [8]:
#Constantes
precioscompra = bd["PrecioUSD"]*TRM
preciosventa = bd["PrecioVenta"]
Costo = 25*TRM

In [9]:
#Añadir las restricciones

compra = np.dot(variables,precioscompra)
r1 = m.add_constraint(compra <= ppto)
if ppto > 3500000:
    r1 = m.add_constraint(sum(apple) >= 1)
    r2 = m.add_constraint(sum(samg) >= 1)
    r3 = m.add_constraint(sum(moto) >= 1)
for i in range(n):
    m.add_constraint(variables[i]<= 5)



In [10]:
#FUNCIÓN OBJETIVO
#Maximizar las ganancias
m.maximize(np.dot(variables,preciosventa) - np.dot(variables,precioscompra))

In [11]:
#Solución
solucion = m.solve()

In [12]:
nombrevar = []
numvariables = []
for v in m.iter_integer_vars():
  numvariables.append(v.solution_value)
  nombrevar.append(v.name)

bd.insert(2,"CantidadOptima", numvariables)

In [13]:
#Calcular impuestos de la Aduana

PreciosUSD = bd[["PrecioUSD","CantidadOptima"]]
PreciosUSD.insert(2,"PrecioCompra",precioscompra)
n2 = len(PreciosUSD)
impuestos=[]

for j in range(n2):
    if PreciosUSD.CantidadOptima[j] != 0:
        if PreciosUSD.PrecioUSD[j] <= 200:
            impuestos.append(PreciosUSD.PrecioUSD[j]*0.2*PreciosUSD.CantidadOptima[j]*TRM)
        else:
            impuestos.append(PreciosUSD.PrecioUSD[j]*0.29*PreciosUSD.CantidadOptima[j]*TRM)
   

Impuesto = sum(impuestos)
#print(Impuesto)

In [ ]:
#Ganancia total
#print(solucion.objective_value)
Ganancia = solucion.objective_value - sum(np.multiply(numvariables,Costo)) - Impuesto
Ganancia = "{:,}".format(round(Ganancia,2)).replace(',','~').replace('.',',').replace('~','.')
print("La Ganancia es = ",Ganancia)
bdfinal = bd[(bd.CantidadOptima!=0)]
bdfinal[["Ref","CantidadOptima","Celular"]]

